In [1]:
import pandas as pd
import numpy as np
import warnings
import time
import requests
from kakao_map import lat_lon
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv("../data/Original_data/train.csv")
test = pd.read_csv("../data/Original_data/test.csv")
xy = pd.read_csv("../data/xy.csv")

print(f"Train Data: {train.shape}")
print(f"Test Data: {test.shape}")
print(f"xy Data: {xy.shape}")

Train Data: (1118822, 52)
Test Data: (9272, 51)
xy Data: (8192, 5)


In [4]:
target = train['target']
train = train.drop(columns=['target'])

train['label'] = 'Train' # train
test['label'] = 'Test' # tets

df = pd.concat([train, test], ignore_index=True)
print(f"df: {df.shape}")
df.head()

df: (1128094, 52)


,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,label
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,Train
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,Train
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,201712,28,5,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,Train
3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,3,4,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,Train
4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,8,2,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,Train


In [5]:
df.rename(columns={'좌표X':'경도', '좌표Y':'위도'}, inplace=True) 
df['구'] = df['시군구'].str.split().str[1]
df['동'] = df['시군구'].str.split().str[2]
cols = ['구', '동'] + [col for col in df.columns if col not in ['구', '동']]
df = df[cols]
print(f"{df[['구', '동']].isna().sum()}")
df.head()

구    0
동    0
dtype: int64


,구,동,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,...,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,경도,위도,단지신청일,label
0,강남구,개포동,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,Train
1,강남구,개포동,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,Train
2,강남구,개포동,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,201712,28,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,Train
3,강남구,개포동,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,3,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,Train
4,강남구,개포동,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,8,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,Train


In [6]:
test = df[df['label'] == 'Test'].drop(columns=['label'])
test.head()

,구,동,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,...,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,경도,위도,단지신청일
1118822,강남구,개포동,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.9700,202307,26,...,직영,4858.00,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0
1118823,강남구,개포동,서울특별시 강남구 개포동,651-1,651.0,1.0,개포더샵트리에,108.2017,202308,15,...,위탁,2724.46,305.0,의무,2022-02-23 13:01:10.0,Y,N,127.056394,37.484892,2022-02-23 11:05:05.0
1118824,강남구,개포동,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,161.0000,202307,28,...,위탁,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0
1118825,강남구,개포동,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,133.4600,202308,10,...,위탁,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0
1118826,강남구,개포동,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,104.4300,202308,18,...,위탁,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0


In [7]:
xy.head()

,구,동,번지,위도,경도
0,강남구,역삼동,762,37.495583,127.046917
1,강북구,수유동,282-20,37.646608,127.014052
2,구로구,구로동,107-4,37.498664,126.891127
3,양천구,신정동,911-4,37.526901,126.857998
4,강서구,등촌동,676,37.549660,126.860381


In [8]:
test_df = test.merge(xy, on=['구', '동', '번지'], how='left', suffixes=('', '_추가'))

mask = test_df['위도'].isna()
test_df.loc[mask, '위도'] = test_df.loc[mask, '위도_추가']
test_df.loc[mask, '경도'] = test_df.loc[mask, '경도_추가']
test_df.drop(columns=['위도_추가', '경도_추가'], inplace=True)

test_df[['위도', '경도']].isna().sum()

위도    6
경도    6
dtype: int64

In [20]:
test_df[test_df[['위도', '경도']].isna().any(axis=1)][['구', '동', '번지', '아파트명', '위도', '경도']].drop_duplicates().reset_index(drop=True)

,구,동,번지,아파트명,위도,경도
0,강남구,역삼동,712-5,역삼센트럴2차아이파크,NaN,NaN
1,관악구,봉천동,1152,관악푸르지오102동,NaN,NaN
2,구로구,항동,가-238,하버라인4단지,NaN,NaN
3,중랑구,상봉동,785,건영2차,NaN,NaN


In [15]:
xy

,구,동,번지,위도,경도
0,강남구,역삼동,762,37.495583,127.046917
1,강북구,수유동,282-20,37.646608,127.014052
2,구로구,구로동,107-4,37.498664,126.891127
3,양천구,신정동,911-4,37.526901,126.857998
4,강서구,등촌동,676,37.549660,126.860381
...,...,...,...,...,...
8187,강남구,삼성동,142-16,37.506294,127.051800
8188,강서구,방화동,890,37.570015,126.821805
8189,광진구,자양동,636-11,37.531539,127.079651
8190,노원구,중계동,512,37.642724,127.066569


In [21]:
# 수동 수집한 위경도 데이터
df_manual_coords = pd.DataFrame({
    '구': ['강남구', '관악구', '구로구', '중랑구'],
    '동': ['역삼동', '봉천동', '항동', '상봉동'],
    '번지': ['712-5', '1152', '가-238', '785'],
    '위도': [37.5000654, 37.4872664, 37.4815438, 37.6011905],
    '경도': [127.0459564, 126.9607512, 126.8260124, 127.0929314]})

df_manual_coords

,구,동,번지,위도,경도
0,강남구,역삼동,712-5,37.500065,127.045956
1,관악구,봉천동,1152,37.487266,126.960751
2,구로구,항동,가-238,37.481544,126.826012
3,중랑구,상봉동,785,37.601191,127.092931


In [22]:
xys = pd.concat([xy, df_manual_coords], ignore_index=True)
xys

,구,동,번지,위도,경도
0,강남구,역삼동,762,37.495583,127.046917
1,강북구,수유동,282-20,37.646608,127.014052
2,구로구,구로동,107-4,37.498664,126.891127
3,양천구,신정동,911-4,37.526901,126.857998
4,강서구,등촌동,676,37.549660,126.860381
...,...,...,...,...,...
8191,중구,만리동2가,176-1,37.554165,126.963448
8192,강남구,역삼동,712-5,37.500065,127.045956
8193,관악구,봉천동,1152,37.487266,126.960751
8194,구로구,항동,가-238,37.481544,126.826012


In [ ]:
df = df.merge(xys, on=['구', '동', '번지'], how='left', suffixes=('', '_추가'))

# 기존 결측치만 수집한 위경도로 채우기
mask = df['위도'].isna()
df.loc[mask, '위도'] = df.loc[mask, '위도_추가']
df.loc[mask, '경도'] = df.loc[mask, '경도_추가']

# 불필요한 보조 컬럼 제거
df.drop(columns=['위도_추가', '경도_추가'], inplace=True)
df[['위도', '경도']].isna().sum()

위도    205
경도    205
dtype: int64

In [ ]:
# 확인
test_df = df[df['label'] == 'Test']
test_df[['위도', '경도']].isna().sum()

위도    0
경도    0
dtype: int64

In [26]:
xys.to_csv("../data/Preprocessed_data/xy.csv", index=False)